## Part 0: Environment Set Up

Run the following cells to load Llama 3.2 1b. These should be the same steps as a3.

In [ ]:
%pip install huggingface_hub
%pip install -U bitsandbytes
!hf auth login

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,StoppingCriteria, StoppingCriteriaList
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", dtype="auto", device_map="auto")

## Part 1: Supervised Fine Tuning

First, load our datasets.

In [ ]:
from datasets import load_dataset

data_files = {
    "train": "/datasets/native_train.jsonl",
    "validation": "/datasets/native_val.jsonl",
    "test": "/datasets/native_test.jsonl"
}

ds = load_dataset("json", data_files=data_files)

train_ds = ds["train"]
val_ds   = ds["validation"]
test_ds  = ds["test"]


Tokenizer and prompt prefix.

In [ ]:
PROMPT = (
    "You are a translation assistant. Translate the Hindi text into English. "
    "Do not add explanations or context. Output only the English translation.\n"
)
